In [1]:
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt

In [ ]:
train = pd.read_csv("zindi_data/Train.csv")

train.sample(10)

In [ ]:
from PIL import Image

images_id = train["Image_ID"].drop_duplicates()
image_shape: pd.Series = images_id.map(lambda x: Image.open("zindi_data/images/"+x).size)

image_shape

In [ ]:
image_shape = image_shape.apply(list).tolist()

image_shape[:5]

In [ ]:
image_shape = pd.DataFrame(image_shape, index=images_id, columns=["width", "height"])

image_shape.sample(5)

In [ ]:
train["class"].value_counts()

In [ ]:
train[
    train["class"] == "NEG"
].head(5)

In [ ]:
train["Image_ID"].nunique()

In [ ]:
test = pd.read_csv("zindi_data/Test.csv")

test.sample(2)

In [ ]:
test["Image_ID"].nunique()

In [ ]:
train[["Image_ID", "class"]].groupby("Image_ID")["class"].nunique()

In [ ]:
train[train["class"] == "NEG"][["Image_ID", "class"]].groupby("Image_ID")["class"].nunique()

In [ ]:
train[
    train["class"] == "NEG"
].describe()

In [ ]:
from glob import glob

len(glob("zindi_data/*.jpg")), 1178 + 2747

In [15]:
from PIL import Image, ImageDraw
import os

cols = ["xmin", "ymin", "xmax", "ymax"]
color_map = {"Trophozoite": "red", "WBC": "blue"}


def draw(image_id: str):
    img = Image.open(os.path.join("zindi_data/images", image_id))
    draw = ImageDraw.Draw(img)
    raw = train[(train["Image_ID"] == image_id) & (train["class"] != "NEG")]
    for _, line in raw.iterrows():
        draw.rectangle([line[i] for i in cols], fill=None, outline=color_map[line["class"]])
    return img

In [ ]:
image_id = train["Image_ID"].sample().values[0]

draw(image_id)

In [ ]:
labels = ["Trophozoite", "WBC", "NEG"]

def prepare(raw: pd.Series):
    x = raw.value_counts().to_dict()
    for i in labels:
        if i not in x:
            x[i] = 0
    return x

train[[
    "Image_ID", "class"
]].groupby("Image_ID")["class"].apply(prepare)

In [ ]:
file_count = train[[
    "Image_ID", "class"
]].groupby("Image_ID")["class"].apply(prepare).reset_index()
file_count

In [ ]:
file_count = file_count.pivot_table(values="class", index="Image_ID", columns="level_1")

file_count

In [ ]:
file_count = file_count.sum(axis=1)
file_count

In [21]:
def split_by_box_count(x):
    if x == 1:
        return 0
    if x <= 5:
        return 1
    if x <= 12:
        return 2
    if x <= 20:
        return 3
    if x <= 30:
        return 4
    return 5

In [ ]:
file_count = file_count.apply(split_by_box_count)

file_count.value_counts()

In [23]:
from sklearn.model_selection import train_test_split

train_image, test_image = train_test_split(file_count.index, test_size=0.1, stratify=file_count.values, random_state=41)

In [ ]:
len(train_image), len(test_image)

In [ ]:
train_image[:5]

In [ ]:
train = pd.merge(train, image_shape.reset_index())

train.sample(5)

In [ ]:
def check_bbox(df):
    # Condition 1: Coordinates must be non-negative
    condition_1 = (df[['ymin', 'xmin', 'ymax', 'xmax']] >= 0).all(axis=1)

    # Condition 2: ymin <= ymax and xmin <= xmax
    condition_2 = (df['ymin'] <= df['ymax']) & (df['xmin'] <= df['xmax'])

    # Condition 3: Bounding box coordinates must be within image bounds
    condition_3 = (df['ymin'] <= df['height']) & (df['ymax'] <= df['height']) & \
                  (df['xmin'] <= df['width']) & (df['xmax'] <= df['width'])

    # Combine all conditions
    return condition_1 & condition_2 & condition_3

condition = check_bbox(train)
condition

In [ ]:
condition.sum()

In [ ]:
print(len(train))
train = train[condition]
print(len(train))

In [ ]:
train_data = train[train["Image_ID"].isin(train_image)]
test_data = train[train["Image_ID"].isin(test_image)]

len(train_data), len(test_data)

In [ ]:
train_data.sample(5)

In [32]:
train_data.to_csv("zindi_data/TrainDataset.csv", index=False)
test_data.to_csv("zindi_data/TestDataset.csv", index=False)